### Pretrained Feature Extractor

In [8]:
import sys
sys.path.insert(0, '../')
from config import *
from dataset import MNIST,CIFAR10, MNIST_SUB

In [64]:
from torchvision.models.feature_extraction import create_feature_extractor
from torchvision.models.feature_extraction import get_graph_node_names
from torchvision.models import resnet50, ResNet50_Weights

weights = ResNet50_Weights.DEFAULT
# I use ResNet for now, but can be changed to other model
# Will use ViT in the future
model = resnet50(weights=weights)

# model.eval()
nodes, _ = get_graph_node_names(model)

In [9]:
ind_idx = [0,2,3,6,8]
ood_idx = [1,7]
# TODO: Use classifier on 1,4,5,7,9, check wass loss
# test_idx = [4,5,9]
mnist_dset_dict = MNIST_SUB(batch_size=128, val_batch_size=64, idx_ind=ind_idx, idx_ood=ood_idx, shuffle=True)

In [10]:
ind_train_loader = mnist_dset_dict['train_set_ind_loader']
ind_val_loader = mnist_dset_dict['val_set_ind_loader']

In [65]:
# print(nodes)
preprocess = weights.transforms()
# Bilinear interpolation is important !
print(preprocess)

ImageClassification(
    crop_size=[224]
    resize_size=[232]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)


In [72]:
x, y = next(iter(ind_train_loader))
# Temporary test
x = torch.repeat_interleave(x, 3, 1) # This is needed just for testing; for CIFAR, ImageNet this is not needed.
# print(preprocess(x))
print(x.shape)

torch.Size([128, 3, 28, 28])


In [67]:
# CAUTION: get rid of the last three layers !
# print(nodes)
print(nodes[:-3])
valid = nodes[:-3]
feat_extractor = create_feature_extractor(
	model, return_nodes=valid)

['x', 'conv1', 'bn1', 'relu', 'maxpool', 'layer1.0.conv1', 'layer1.0.bn1', 'layer1.0.relu', 'layer1.0.conv2', 'layer1.0.bn2', 'layer1.0.relu_1', 'layer1.0.conv3', 'layer1.0.bn3', 'layer1.0.downsample.0', 'layer1.0.downsample.1', 'layer1.0.add', 'layer1.0.relu_2', 'layer1.1.conv1', 'layer1.1.bn1', 'layer1.1.relu', 'layer1.1.conv2', 'layer1.1.bn2', 'layer1.1.relu_1', 'layer1.1.conv3', 'layer1.1.bn3', 'layer1.1.add', 'layer1.1.relu_2', 'layer1.2.conv1', 'layer1.2.bn1', 'layer1.2.relu', 'layer1.2.conv2', 'layer1.2.bn2', 'layer1.2.relu_1', 'layer1.2.conv3', 'layer1.2.bn3', 'layer1.2.add', 'layer1.2.relu_2', 'layer2.0.conv1', 'layer2.0.bn1', 'layer2.0.relu', 'layer2.0.conv2', 'layer2.0.bn2', 'layer2.0.relu_1', 'layer2.0.conv3', 'layer2.0.bn3', 'layer2.0.downsample.0', 'layer2.0.downsample.1', 'layer2.0.add', 'layer2.0.relu_2', 'layer2.1.conv1', 'layer2.1.bn1', 'layer2.1.relu', 'layer2.1.conv2', 'layer2.1.bn2', 'layer2.1.relu_1', 'layer2.1.conv3', 'layer2.1.bn3', 'layer2.1.add', 'layer2.1.rel

In [70]:
feat_extractor.requires_grad_(False)
out = feat_extractor(x)
print(out['layer4.2.relu_2'].shape)
print(valid[-1])
print(out[valid[-1]].shape)

torch.Size([128, 2048, 1, 1])
layer4.2.relu_2
torch.Size([128, 2048, 1, 1])
